In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv', index_col = 'RowNumber')
dataset.head()


,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
#Customer ID and Surname would not be relevant as features
X_columns = dataset.columns.tolist()[2:12]
Y_columns = dataset.columns.tolist()[-1:]
print(X_columns)
print(Y_columns)

['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
['Exited']


In [4]:
X = dataset[X_columns].values 
Y = dataset[Y_columns].values

In [5]:
#We need to encode categorical variables such as geography and gender
from sklearn.preprocessing import LabelEncoder
X_column_transformer = LabelEncoder()
X[:, 1] = X_column_transformer.fit_transform(X[:, 1])

In [6]:
#Lets Encode gender now
X[:, 2] = X_column_transformer.fit_transform(X[:, 2])

We are treating countries with ordinal values(0 < 1 < 2) but they are incomparable.
To solve this we can use one hot encoding.
We will perform some standardization 

In [7]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

pipeline = Pipeline(
    [
        ('Categorizer', ColumnTransformer(
            [
                ("Gender Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [2]),
                ("Geography Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [1])
            ], 
            remainder = 'passthrough', n_jobs = 1)),
        ('Normalizer', StandardScaler())
    ]
)

In [8]:
#Standardize the features
X = pipeline.fit_transform(X)

In [9]:
#Spilt the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [10]:
#Let us create the Neural Network
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [11]:
#Initialize ANN
classifier = Sequential()

In [12]:
#Add input layer and hidden layer
classifier.add(Dense(6, activation = 'relu', input_shape = (X_train.shape[1], )))
classifier.add(Dropout(rate = 0.1))

In [13]:
#Add second layer
classifier.add(Dense(6, activation = 'relu'))
classifier.add(Dropout(rate = 0.1))

In [14]:
#Add output layer
classifier.add(Dense(1, activation = 'sigmoid'))

In [15]:
#Let us take a look at our network
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 72        
_________________________________________________________________
dropout (Dropout)            (None, 6)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dropout_1 (Dropout)          (None, 6)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [16]:
#Optimize the weights
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [17]:
#Fitting the Neural Network
history = classifier.fit(X_train, y_train, batch_size = 32, epochs = 200, validation_split = 0.1, verbose = 2)

Epoch 1/200
225/225 - 15s - loss: 0.6117 - accuracy: 0.7186 - val_loss: 0.5217 - val_accuracy: 0.7925
Epoch 2/200
225/225 - 0s - loss: 0.4991 - accuracy: 0.7969 - val_loss: 0.4626 - val_accuracy: 0.7950
Epoch 3/200
225/225 - 0s - loss: 0.4719 - accuracy: 0.8001 - val_loss: 0.4423 - val_accuracy: 0.8000
Epoch 4/200
225/225 - 0s - loss: 0.4587 - accuracy: 0.8022 - val_loss: 0.4284 - val_accuracy: 0.8037
Epoch 5/200
225/225 - 0s - loss: 0.4462 - accuracy: 0.8069 - val_loss: 0.4182 - val_accuracy: 0.8062
Epoch 6/200
225/225 - 0s - loss: 0.4419 - accuracy: 0.8099 - val_loss: 0.4113 - val_accuracy: 0.8050
Epoch 7/200
225/225 - 0s - loss: 0.4345 - accuracy: 0.8125 - val_loss: 0.4041 - val_accuracy: 0.8125
Epoch 8/200
225/225 - 0s - loss: 0.4258 - accuracy: 0.8161 - val_loss: 0.3973 - val_accuracy: 0.8163
Epoch 9/200
225/225 - 0s - loss: 0.4219 - accuracy: 0.8210 - val_loss: 0.3915 - val_accuracy: 0.8213
Epoch 10/200
225/225 - 0s - loss: 0.4170 - accuracy: 0.8232 - val_loss: 0.3869 - val_accur

In [18]:
y_pred = classifier.predict(X_test)
print(y_pred[:5])

[[0.32293567]
 [0.31686765]
 [0.22650653]
 [0.11121964]
 [0.07013544]]


In [19]:
#Let us use confusion matrix with cutoff value as 0.5
y_pred = (y_pred > 0.5).astype(int)
print(y_pred[:5])

[[0]
 [0]
 [0]
 [0]
 [0]]


In [20]:
#Making the Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1554   41]
 [ 232  173]]


In [21]:
#Accuracy of our NN
print(((cm[0][0] + cm[1][1])* 100) / len(y_test), '% of data was classified correctly')

86.35 % of data was classified correctly
